In [1]:
print("Importing packages...")
#Importando pacotes
import numpy as np
import pandas as pd
from argparse import ArgumentParser
import subprocess
import pandas as pd
from joblib import Parallel, delayed
from IPython.display import clear_output, display
from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file

import multiprocessing
import time
import re
import os

Importing packages...


Using TensorFlow backend.


In [2]:
print("Loading data...")
print("1/3...")
ids_achados    = pd.read_csv('../../data/resultados/greenZoneIds.csv', index_col = 0)
print("2/3...")
dados_texto    = pd.read_csv('../../data/DATA_LAUDOS_TEXTO_formato1', sep = ";")
print("3/3...")
labels_file    = pd.read_csv("../../data/resultados/resultLabels.csv", index_col = 0)

/home/speed/joao.pedrosa/miniconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
print("Terraforming data...")
print("1/2...")
dados_treino    = dados_texto[dados_texto['ID_EXAME'].isin(ids_achados['id'])]
# dados_validacao = dados_texto[dados_texto['ID_EXAME'].isin(ids_achados['id'][-400000:])]
labels_treino   = labels_file[labels_file["idExame"].isin(ids_achados['id'])]
# labels_validacao    = labels_file[labels_file["idExame"].isin(ids_achados['id'][-400000:])]

In [4]:
#max number of words in embedding
embed_dim = 512
lstm_out_dim = 256
top_words = 7500

text   = dados_treino['CONTEUDO']
labels = labels_treino['label']
labels = [array[1:-1].split(",") for array in labels]
labels = [[1 if (value == 'True' or value == ' True') else 0 for value in label] for label in labels]
labels = np.array(labels)

class_dim = 3
k = 3

In [12]:
print("2/2...")
#Tokenizing
tokenizer = Tokenizer(num_words = top_words, split = ' ')
tokenizer.fit_on_texts(text)
X = tokenizer.texts_to_sequences(text)

KeyboardInterrupt: 

In [ ]:
print("Building model...")
#number of iteractions
epochs = 30 
batch_size = 128

shape = pad_sequences(X).shape[1]
model = Sequential()
model.add(Embedding(top_words, embed_dim,input_length = shape))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out_dim, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(74, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
#Training the model
X = pad_sequences(X)

In [ ]:
print("Training model...")
model.fit(X, labels, epochs = epochs, batch_size=batch_size, verbose = 2)

In [ ]:
print("Saving model...")
model.save('../../data/resultados/modeloClassificacao.h5')
print("Done!")